In [ ]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [ ]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        df = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~df["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

df


We can use the `<=>` operator to run vector search within the database

In [ ]:

vec = json.loads(df.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

In [ ]:
# Define a function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    # No need to convert to np.array as the inputs are already numpy arrays
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine

# Create the database engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Query to load the auctions table
auctions_query = 'SELECT * FROM auctions;'
auctions_df = pd.read_sql(auctions_query, con=engine)

# Query to load the app_vectors table
vectors_query = 'SELECT * FROM app_vectors;'
vectors_df = pd.read_sql(vectors_query, con=engine)

# Determine if embeddings are ready in the vectors_df
has_embedding = (~vectors_df["embedding"].isna()).all()
if not has_embedding:
    print("Waiting for embeddings...")
    # You might need to loop and check periodically if embeddings are ready
    # or handle this logic according to your application's requirements



In [ ]:
# Prepare embeddings
embeddings = [np.array(json.loads(e)) for e in vectors_df['embedding']]

# Initialize similarity matrix
similarity_matrix = np.zeros((len(embeddings), len(embeddings)))

# Calculate the similarity matrix
for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        similarity = cosine_similarity(embeddings[i], embeddings[j])
        similarity_matrix[i, j] = similarity_matrix[j, i] = similarity

# Fill the diagonal of the similarity matrix with 1s
np.fill_diagonal(similarity_matrix, 1)

# Now similarity_matrix is the cosine similarity matrix for the vectors

# Convert the similarity matrix to a DataFrame for easier processing
similarity_df = pd.DataFrame(similarity_matrix, index=vectors_df['bundleId'], columns=vectors_df['bundleId'])
# Calculate the average similarity 
average_similarity = similarity_matrix.mean(axis=1)

vectors_df['average_similarity'] = average_similarity

merged_df = pd.merge(auctions_df, vectors_df[['bundleId', 'average_similarity']], on='bundleId', how='inner')
# Calculate the correlation
correlation = merged_df[['average_similarity', 'sentPrice']].corr().iloc[0, 1]
print("Correlation between average description similarity and sentPrice:", correlation)


In [ ]:
average_similarity
similarity_df.head()

In [ ]:
vectors_df.head()

In [ ]:
# correlation between the average similarity and sentPrice in the merged dataframe
correlation_with_price = merged_df[['average_similarity', 'sentPrice']].corr().iloc[0, 1]
print("Correlation with sentPrice:", correlation_with_price)

# To check for consistency with other features, calculate the correlation for each feature
for feature in ['countryCode', 'osAndVersion']:
    merged_df[f'average_similarity_{feature}'] = merged_df.groupby(feature)['average_similarity'].transform('mean')
    correlation_with_feature = merged_df.groupby(feature).apply(
        lambda x: x[['average_similarity', 'sentPrice']].corr().iloc[0, 1]
    )
    print(f"Correlation with {feature}:", correlation_with_feature.mean())


In [ ]:
# pairwise_correlations = []
# for i in range(len(merged_df)):
#     for j in range(i+1, len(merged_df)):
#         price_difference = abs(merged_df.loc[i, 'sentPrice'] - merged_df.loc[j, 'sentPrice'])
#         description_similarity = merged_df.loc[i, 'average_similarity']
#         pairwise_correlations.append((description_similarity, price_difference))

# # Convert to DataFrame
# pairwise_df = pd.DataFrame(pairwise_correlations, columns=['description_similarity', 'price_difference'])

# # Calculate correlation
# pairwise_correlation = pairwise_df.corr().iloc[0, 1]
# print("Pairwise correlation between description similarity and price difference:", pairwise_correlation)


In [ ]:
#so all rows are too long to compute..so lets try Binning Similarity Scores
# by groupi the similarity scores into bins (e.g., high, medium, low similarity) and analyze the variation in sentPrice within these bins instead of using a continuous similarity measure.

# for simplicity:
df = merged_df
# Bin the average similarity scores into quantiles
try:
    df['similarity_bin'] = pd.qcut(df['average_similarity'], q=10, labels=False, duplicates='drop')
except ValueError as e:
    print("ValueError:", e)
    # If an error occurs (e.g., due to too few unique quantiles), handle it accordingly
    # For example, you might decide to use fewer quantiles
    df['similarity_bin'] = pd.qcut(df['average_similarity'], q=5, labels=False, duplicates='drop')

# Group by the similarity bin and calculate the mean sentPrice for each bin
mean_sent_price_by_bin = df.groupby('similarity_bin')['sentPrice'].mean()

# Now, see if there's a trend in the mean sentPrice across the bins
trend = mean_sent_price_by_bin.sort_index()

# Optionally, perform a polynomial fit to see the trend more clearly
coefficients = np.polyfit(trend.index, trend.values, deg=1)
poly_fit = np.poly1d(coefficients)

# Print the coefficients or use them to plot the trend line
print(f"Trend line coefficients: {coefficients}")


We got a slope coefficient of 0.09433349 and an intercept of 0.92230247 from our trend line analysis. Let's break down what this means for our dataset:

Slope (0.09433349): This number tells us about the relationship between the similarity scores and sentPrice. A positive slope like the one we got, approximately 0.094, indicates a slight upward trend. This suggests that apps with higher similarity scores might be associated with a modestly higher sentPrice on average.

Intercept (0.92230247): This value predicts the sentPrice for the baseline case where the similarity score bin is zero. So, if an app's description had the lowest level of similarity compared to others (assuming the lowest bin is coded as zero), our model would predict its sentPrice to be around 0.922.

The equation we've derived for the trend line is sentPrice = 0.09433349 * similarity_bin + 0.92230247, giving us a linear model to estimate sentPrice from our binned similarity scores.

We should approach these results with a degree of caution. The coefficients provide an estimate but don't account for the variability of the data or the significance of the trend. To better understand the model's accuracy, we'd need to look at the R-squared value or conduct hypothesis testing on the coefficients to see if they're statistically significant and not due to random chance.

In [ ]:
def robust_regression(X, y, num_iterations=5, k=4.685):
    weights = np.ones(len(y))
    for iteration in range(num_iterations):
        # Apply weights to X and y
        XW = X * np.sqrt(weights[:, np.newaxis])
        yW = y * np.sqrt(weights)
        
        # Solve the weighted least squares problem
        beta = np.linalg.lstsq(XW, yW, rcond=None)[0]
        
        # Calculate residuals and update weights
        residuals = y - X.dot(beta)
        mad = np.median(np.abs(residuals - np.median(residuals)))
        adjusted_residuals = np.sqrt(np.abs(residuals)) / (mad + np.finfo(float).eps)
        weights = (1 - (adjusted_residuals / k)**2)**2
        weights[adjusted_residuals >= k] = 0
    return beta

results = []  # Initialize the results list

x = df['average_similarity'].values
y = df['sentPrice'].values

for degree in range(2, 11):
    # Generate polynomial features
    X_poly = np.vander(x, degree + 1, increasing=True)
    
    # Apply robust regression to the polynomial features
    beta_poly = robust_regression(X_poly, y)
    
    # Predict y using the model
    y_pred = X_poly.dot(beta_poly)
    
    # Calculate residuals
    residuals = y - y_pred
    
    # Calculate R-squared
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    
    results.append((degree, r_squared))
    
    # Print the results for each polynomial degree
    print(f"Degree: {degree}, R-squared: {r_squared}")
    print(f"Degree: {degree}, Coefficients: {beta_poly}")
    print("-" * 30)


The R-squared values we got are negative across all polynomial degrees tested, from 2 through 10. Typically, an R-squared value aims to measure the proportion of variance in the dependent variable that is predictable from the independent variable(s). Negative R-squared values suggest that our polynomial models fit the data worse than a simple horizontal line at the mean of sentPrice. This implies that none of the models provide a satisfactory explanation for the variance in our target variable.

As for the significance of the model coefficients (the beta values for each polynomial degree), determining their significance usually involves examining p-values from hypothesis tests, such as the t-test, for each coefficient to assess if they significantly differ from zero. Without conducting these tests and calculating p-values, we can't directly assess the significance of the coefficients from the provided results.

The outcomes of our robust regression and polynomial modeling efforts indicate that these approaches have not yielded a model that effectively captures the relationship between our predictor(s) and the target variabhis:

Given our dataset with over 41,000 rows, we opted not to implement the k-nearest neighbors (KNN) regression, a form of non-parametric regression. While KNN regression can be quite intuitive and straightforward, involving averaging the target values of the k nearest points in the feature space for each prediction point, it faces significant scalability challenges, especially with large datasets like ours.

The primary reason for this decision is the computational complexity associated with KNN. For each prediction, KNN requires calculating the distance from the prediction point to all other points in the dataset, identifying the k nearest neighbors based on these distances, and then averaging their target values. This process becomes increasingly time-consuming as the dataset grows, making it impractical for datasets with tens of thousands of observations or more.

Additionally, while KNN is a flexible modeling approach that can capture complex relationships without assuming a specific functional form, it lacks the statistical rigor of more advanced non-parametric methods, such as kernel regression or LOESS (Locally Estimated Scatterplot Smoothing). These methods involve more sophisticated techniques for estimating the regression function, which can provide a better fit to the data but also require significant computational resources.

Here's a brief overview of how KNN regression could be implemented:

In [ ]:
def knn_regression(predict_point, features, targets, k):
    # Calculate distances from predict_point to all other points
    distances = np.linalg.norm(features - predict_point, axis=1)
    
    # Find the indices of the k nearest neighbors
    k_indices = np.argpartition(distances, k)[:k]
    
    # Average the target values of the nearest neighbors
    return np.mean(targets[k_indices])

While the KNN approach remains a valuable tool for certain applications, especially when dealing with smaller datasets or when computational resources are not a constraint, it's not the most efficient choice for our current scenario. The computational demands and time complexity issues associated with applying KNN to our large dataset led us to explore alternative modeling strategies that could offer a more practical balance between model performance and computational efficiency.

we can try exploring more complex relationships, such as how the bidFloorPrice might interact with the similarity scores and affect the sentPrice, is a valuable next step. This approach allows us to investigate whether there's an interaction effect that could explain the variance in sentPrice more effectively. By considering bidFloorPrice in our model, we're essentially looking to see if the relationship between app description similarity and sentPrice changes at different levels of bidFloorPrice.

In [ ]:
bidFloorPrice = df['bidFloorPrice'].values

# Let's consider a simple model with a quadratic term for similarity and its interaction with bidFloorPrice
X = np.vstack([
    np.ones(len(x)),  # Intercept
    x,  # Linear term for similarity
    x**2,  # Quadratic term for similarity
    bidFloorPrice,  # Linear term for bidFloorPrice
    x * bidFloorPrice,  # Interaction term
]).T

# Fit the model
beta = np.linalg.lstsq(X, y, rcond=None)[0]

# Predictions
y_pred = X.dot(beta)

# Calculate residuals
residuals = y - y_pred

# Compute R-squared
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

print(f"Model coefficients: {beta}")
print(f"R-squared: {r_squared}") 		

Model Coefficients: The model coefficients for our regression model are [30.43419184, -125.00554341, 130.34515304, 490.4529712, -874.71697674]. These coefficients correspond to the intercept, the linear term for similarity scores, the quadratic term for similarity scores, the linear term for bidFloorPrice, and the interaction term between similarity scores and bidFloorPrice, respectively.

R-squared Value: The R-squared value is 0.0152058833363653. This value indicates that the model explains approximately 1.52% of the variance in the sentPrice.

In [ ]:
# Convert 'eventTimestamp' to numeric (float or int), then convert from milliseconds to seconds
df['eventTimestamp'] = pd.to_numeric(df['eventTimestamp']) / 1000

# Now convert 'eventTimestamp' to datetime format
df['eventTimestamp'] = pd.to_datetime(df['eventTimestamp'], unit='s')

# Proceed with extracting time-based features
df['hour'] = df['eventTimestamp'].dt.hour
df['day_of_week'] = df['eventTimestamp'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Add an intercept and other predictors to X as needed
X = df[['hour', 'is_weekend']]  # You can add more features based on your analysis needs
y = df['sentPrice']

# Add an intercept
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Fit the model
beta = np.linalg.lstsq(X, y, rcond=None)[0]

# Model evaluation: Calculate predictions and R-squared
y_pred = X.dot(beta)
residuals = y - y_pred
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

print(f"Model coefficients: {beta}")
print(f"R-squared: {r_squared}")


The R-squared value of 0.00026574580231197853 is very close to zero, indicating that our model explains only a tiny fraction of the variance in sentPrice. This suggests that the features we've included (hour and is_weekend) do not strongly predict sentPrice on their own.

lets try to analyze the impact of the 'osAndVersion' column, which is a string indicating the operating system and its version, could provide insights into how different OS versions affect sentPrice. Since osAndVersion is categorical, we would first need to convert it into a format suitable for regression analysis. This typically involves creating dummy variables (also known as one-hot encoding) for each category.

In [ ]:
# Convert 'osAndVersion' into dummy variables, automatically dropping the first category
os_version_dummies = pd.get_dummies(df['osAndVersion'], prefix='os_version', drop_first=True)

# Assuming 'sentPrice' is your target variable
y = df['sentPrice'].values

# If you have other features you want to include, prepare them
# Here, we'll proceed with only the 'os_version' dummies for demonstration purposes
# Make sure to replace 'X_other' with actual other features if available
X_other = pd.DataFrame()  # This is a placeholder; replace with your actual other features

# Directly use the dummies as part of your features matrix if not joining back to 'df'
X = os_version_dummies.values

# Assuming X_other is not just an empty DataFrame, you would concatenate it like so:
# X = np.hstack([X_other.values, X])

# Fit the model
# Adding an intercept to X
X = np.hstack([np.ones((X.shape[0], 1)), X])

beta = np.linalg.lstsq(X, y, rcond=None)[0]

# Model evaluation: Calculate predictions and R-squared
y_pred = X.dot(beta)
residuals = y - y_pred
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

print(f"Model coefficients: {beta}")
print(f"R-squared: {r_squared}")


An R-squared value of 0.010466412999634023 means that approximately 1.05% of the variability in sentPrice is explained by the OS and version categories. While we have successfully included the osAndVersion variable in our model, the low R-squared value indicates that this model, as it stands, explains only a small fraction of the variance in sentPrice.

Considerations:
Complexity vs. Insight: The incorporation of many dummy variables for osAndVersion adds complexity to the model. Although we now have insights into how different OS versions might affect sentPrice, the overall explanatory power of the model remains low.
Potential for Overfitting: With a large number of coefficients relative to the amount of data, there's a risk of overfitting, especially when the R-squared is low. It's essential to validate the model on a separate test set or use cross-validation techniques to assess its predictive performance.
Further Exploration Needed: The low R-squared value suggests that additional factors not captured by the osAndVersion alone significantly influence sentPrice. It might be beneficial to explore other variables, interaction terms, or even non-linear models to improve the model's explanatory power.
Domain Knowledge and Feature Engineering: Further domain knowledge could inform more nuanced feature engineering, such as grouping similar OS versions or extracting broader categories (e.g., iOS vs. Android) that might have more substantial effects on sentPrice.

Significant coefficients, especially those with large magnitudes, do indicate that specific OS versions have a stronger relationship with sentPrice than others. Analyzing these coefficients can provide insights into which OS versions are associated with higher or lower prices, potentially pointing to preferences or trends in the dataset that could inform strategic decisions:

In [ ]:
df.head()

In [ ]:
# Lets calculate the R-squared value manually:
x = df['similarity_bin'].values
y = df['sentPrice'].values
y_pred = coefficients[0] * x + coefficients[1]  # The predicted sentPrice values

# Calculate R-squared manually
ss_res = np.sum((y - y_pred) ** 2)  # Sum of squares of residuals
ss_tot = np.sum((y - np.mean(y)) ** 2)  # Total sum of squares
r_squared = 1 - (ss_res / ss_tot)

print(f"R-squared value: {r_squared}")

In [ ]:
# Predicted values
y_pred = coefficients[0] * x + coefficients[1]

# Residuals (errors) between actual and predicted values
residuals = y - y_pred

# Sum of squared residuals
ss_res = np.sum(residuals**2)

# Sample variance of the residuals (mean square error)
mse = ss_res / (len(y) - 2)

# Standard error of coefficients
X = np.vstack([x, np.ones(len(x))]).T  # Independent variable matrix with a column of ones for the intercept
var_beta = mse * np.linalg.inv(np.dot(X.T, X)).diagonal()  # Variance of coefficients
se_beta = np.sqrt(var_beta)  # Standard error of coefficients
t_stats = coefficients / se_beta
# Calculate standard error for each coefficient
# We assume 'X' is your matrix of independent variables
X = np.vstack([np.ones(len(x)), x]).T  # Add a column of ones for the intercept
XtX_inv = np.linalg.inv(X.T.dot(X))  # Calculate (X^T * X)^(-1)
mse = np.mean((y - y_pred) ** 2)  # Mean squared error
se = np.sqrt(np.diagonal(mse * XtX_inv))  # Standard error of coefficients

# Calculate t-statistics
t_stats = coefficients / se

print("T-statistics for the coefficients:", t_stats)

The t-test is appropriate for hypothesis testing in regression analysis when you are trying to determine if there is a statistically significant relationship between the independent and dependent variables. It tests whether the coefficients in a regression model are significantly different from zero in a sample.

However, your output indicates an R-squared value that is negative, which suggests that the model fits the data worse than a horizontal line at the mean of the dependent variable (sentPrice). This could happen when the predictions are worse than just predicting the mean value of sentPrice, and it implies that the model might not be appropriate foourur dat
a.

For the slope (1.5945055): If this t-statistic corresponds to a p-value below a certain significance level (commonly 0.05), it indicates that the slope is significantly different from zero. The value of 1.59 is on the lower side for significance, and without the corresponding p-value or critical t-value, it’s not possible to definitively say whether it's significant.

For the intercept (52.64044185): This high t-statistic suggests the intercept is significantly different from zero, implying that the baseline level of sentPrice when the similarity_bin is zero is significantly above zero.

Given that the R-squared value is effectively zero (or negative), even if the t-statistics indicate that coefficients are statistically significant, the model explains none of the variability of the dependent variable around its mean. This model is not useful for predictive purposes or possibly even for inference about the relationship between the variables.

In [ ]:
#lets fit our Model with NumPy:
beta = np.linalg.inv(X.T @ X) @ X.T @ y
y_pred = X @ beta
residuals = y - y_pred
sigma_squared = np.sum(residuals**2) / (X.shape[0] - X.shape[1])
cov_matrix = sigma_squared * np.linalg.inv(X.T @ X)
std_errors = np.sqrt(np.diag(cov_matrix))
t_stats = beta / std_errors
alpha = 0.05
t_critical_approx = 1.96  # Approximation for large df
conf_intervals = [(b - t_critical_approx * se, b + t_critical_approx * se) for b, se in zip(beta, std_errors)]

print("Coefficients (beta):", beta)
print("\nStandard Errors:", std_errors)
print("\nT-Statistics:", t_stats)

# Printing confidence intervals
print("\n95% Confidence Intervals:")
for idx, ci in enumerate(conf_intervals):
    print(f"Variable {idx}: {ci}")

# Note on the critical value approximation
print(f"\nNote: Used a critical value approximation of {t_critical_approx} for large degrees of freedom.")


In our scenario, we conducted a linear regression analysis to investigate the hypothesis that apps with similar descriptions, as represented by their embeddings, would have similar asking prices in auctions. After calculating the regression coefficients (beta), standard errors, t-statistics, and confidence intervals using NumPy, here's what we found:

Coefficients (Beta): We obtained two coefficients: [1.1407896, 0.06927775]. The first coefficient corresponds to the intercept, and the second to the slope of our regression line. These coefficients suggest a relationship between the average similarity of app descriptions and their asking prices in auctions.

Standard Errors: The standard errors for these coefficients are [0.0591484, 0.01751689]. These values help us understand the precision of our coefficient estimates. Lower standard errors indicate more precise estimates.

T-Statistics: The t-statistics [19.28690588, 3.95491223] indicate how many standard deviations our coefficients are from 0. These values are used to assess the statistical significance of the coefficients.

95% Confidence Intervals:

For the intercept: (1.024858735445414, 1.2567204579809839)
For the slope: (0.03494465353206231, 0.10361085339321854)
These intervals give us a range within which we can be 95% confident that the true value of our coefficients lies.

Given the t-statistics and confidence intervals, we observe that both coefficients seem significant, indicating a relationship between the similarity of app descriptions and their asking prices. However, it's crucial to note that these results are based on an approximation using a critical value of 1.96, typically applied when degrees of freedom are large.

To further validate our findings and explore the relationships in our data, we could consider:

Cross-validation: Split our data into training and test sets to evaluate the model's performance on unseen data. This can help us assess the model's predictive accuracy and generalizability.

Exploring Additional Features: Our current model focuses on the similarity of app descriptions. Incorporating other features from the auctions table (like CountryCode and OS) might provide a more comprehensive understanding of what influences asking prices.

Interaction Effects: Examining interaction terms between app description similarity and other variables (e.g., OS version or country) might reveal more complex relationships affecting auction prices.

Model Complexity: If our initial model doesn't capture the nuances of our data well, we might explore more complex models or non-linear relationships. Polynomial regression or even machine learning models could offer deeper insights, provided we carefully manage model complexity to avoid overfitting.

Statistical Tests for Categorical Variables: If we incorporate categorical variables like OS, using ANOVA or similar statistical tests could help assess the overall significance of these categories on auction prices, complementing the individual coefficient analysis from regression.

By following these steps, we aim to refine our analysis, potentially uncovering more nuanced insights into the factors that influence the auction prices of mobile app advertisements.

lets start with cross validation:

In [ ]:
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Number of observations
n = X.shape[0]

# Shuffle the dataset (optional but recommended)
indices = np.arange(n)
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Splitting the dataset into training and testing sets
split_ratio = 0.8  # 80% of the data used for training, 20% for testing
split_index = int(n * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Fit the model on the training data using pseudoinverse
beta = np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ y_train

# Predict on the testing data
y_pred = X_test @ beta

# Calculate the Mean Squared Error (MSE) on the test set
mse = np.mean((y_test - y_pred) ** 2)
# R-squared calculation
ss_res = np.sum((y_test - y_pred) ** 2)
ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# MAE calculation
mae = np.mean(np.abs(y_test - y_pred))

print(f"R-squared on Test Set: {r_squared}")
print(f"Mean Absolute Error (MAE) on Test Set: {mae}")
print(f"Mean Squared Error on Test Set: {mse}")



The results from our model evaluation on the test set offer insights into its performance and predictive accuracy:

R-squared on Test Set: 0.00021489117968065408: This value is very close to zero, indicating that the model explains a very small fraction of the variance in the dependent variable (auction prices) based on the independent variables (features like app description similarity, OS version, etc.) in the test set. Essentially, this suggests that the model, as currently specified, has limited predictive power and does not capture the underlying relationship between the features and auction prices effectively.

Mean Absolute Error (MAE) on Test Set: 1.911010887864504: The MAE provides an average of the absolute differences between predicted and actual auction prices. A MAE of approximately 1.91 means that on average, the model's predictions deviate from the actual prices by about 1.91 units. The scale and acceptability of this error depend on the context of your problem and the range of sentPrice in your dataset.

Mean Squared Error (MSE) on Test Set: 32.726211377151074: The MSE is higher than the MAE, as expected, because it squares the errors before averaging them, thus giving a larger weight to larger errors. An MSE of approximately 32.73 suggests that there are significant deviations between the predicted and actual prices, reinforced by the fact that squaring the errors amplifies the impact of larger errors on this metric.

For us, these metrics indicate that while we have developed a regression model to predict auction prices based on various features, the model's current formulation does not strongly predict auction prices. The very low R-squared value suggests that other unaccounted factors may be influencing auction prices, or that the relationships between the features and auction prices are more complex than our model captures.

Now, let's address the second question about examining interaction terms, specifically in the context of "app description similarity and other variables (e.g., OS version or country)" to uncover complex relationships that might affect auction prices. To include interaction terms in your regression model, you'd manually create these terms and add them to your feature matrix. Here’s an example:

In [ ]:

# Let's create a simple manual label encoder for 'os_version'
unique_os_versions = df['osAndVersion'].unique()
os_version_to_int = {key: value for value, key in enumerate(unique_os_versions)}

# Map the categorical 'os_version' to integers
df['os_version_encoded'] = df['osAndVersion'].map(os_version_to_int)

# Now, 'df' contains a new column 'os_version_encoded' with numeric representations of the OS versions
os_version_encoded= df['os_version_encoded']
# Manually create an interaction term
interaction_term = similarity * os_version_encoded

# Convert the interaction_term pandas Series to a numpy array and reshape
interaction_term_array = interaction_term.values.reshape(-1, 1)

# X = np.hstack([X, interaction_term_array])
# beta = np.linalg.inv(X.T @ X) @ X.T @ y

# y_pred = X @ beta
# residuals = y - y_pred
# y_pred = X @ beta
# SS_res = np.sum(residuals**2)
# SS_tot = np.sum((y - np.mean(y))**2)
# R_squared = 1 - (SS_res / SS_tot)
# print(f"R-squared: {R_squared}")

# Replace the direct inversion and matrix multiplication with numpy.linalg.lstsq
# X = np.hstack([X, interaction_term_array])  # Assuming this is correctly shaped
# Add a column of ones if you haven't already to include the intercept
X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])

# Use np.linalg.lstsq to find the beta coefficients
beta, residuals, rank, s = np.linalg.lstsq(X_with_intercept, y, rcond=None)

# Predict y using the beta coefficients
y_pred = X_with_intercept @ beta

# Calculate residuals
residuals = y - y_pred

# Now, you can proceed with further analysis like calculating R-squared
RSS = np.sum(residuals**2)  # Residual Sum of Squares
TSS = np.sum((y - np.mean(y))**2)  # Total Sum of Squares
R_squared = 1 - RSS / TSS

print(f"R-squared: {R_squared}")



Receiving an R-squared value of approximately 0.0004 indicates that the model, including the interaction term you've added, explains a very small fraction of the variance in the dependent variable (the auction price in this context). Here's what this could mean in the context of our data and steps we've taken:

Interpretation:
Minimal Variance Explained: The R-squared value suggests that the combination of features and the interaction term you've included in the model does not significantly explain the variability in auction prices. Essentially, the model is only slightly better than a simple model that always predicts the mean auction price, regardless of the input features.
Potential Overfitting: While the R-squared is low here, in cases where it's significantly higher, one should also consider the possibility of overfitting, especially when adding interaction terms or many predictors to a model. However, in this case, the concern is not overfitting but underfitting, where the model fails to capture the underlying relationship.

Model Complexity with Polynomial Regression
Polynomial regression allows you to model non-linear relationships between the independent variables and the dependent variable by introducing polynomial terms (squared, cubed, etc.) of the independent variables. This can capture more complex patterns in your data but also increases the risk of overfitting, especially as the degree of the polynomial increases.

In [ ]:

# Let's create a 2nd degree polynomial feature for demonstration
x_squared = x ** 2

# Now, prepare the design matrix X with the original and the squared term
X_poly = np.vstack([np.ones(len(x)), x, x_squared]).T

# Fit the polynomial regression model
beta_poly = np.linalg.inv(X_poly.T @ X_poly) @ X_poly.T @ y

# Predictions
y_pred_poly = X_poly @ beta_poly

# Calculate residuals
residuals_poly = y - y_pred_poly

# Calculate the Total Sum of Squares (TSS)
TSS = np.sum((y - np.mean(y))**2)

# Calculate the Residual Sum of Squares (RSS)
RSS = np.sum(residuals_poly**2)

# Calculate R-squared
R_squared_poly = 1 - (RSS / TSS)

# Printing the results
print(f"R-squared for Polynomial Regression Model: {R_squared_poly:.3f}")
print(f"Residuals: {residuals_poly[:10]}")  # Printing the first 10 residuals as an example



R-squared for Polynomial Regression Model: 0.000: This indicates that the polynomial regression model explains none of the variance in the dependent variable around its mean. An R-squared value of 0 suggests that the model does not improve the prediction over simply using the mean of the dependent variable as the prediction for all observations. In practical terms, this means the model, as currently specified, likely does not capture the relationship between your predictors and the outcome effectively.

Residuals: The list of residuals represents the difference between the actual values of your dependent variable and the values predicted by your model for the first 10 observations. For example, a residual of -1.30098274 for the first observation indicates that the model's prediction for this particular data point was higher than the actual value by approximately 1.3 units.

Interpreting These Results
Low Predictive Power: The near-zero R-squared value suggests that the model's predictive power is minimal, and it might not be capturing the necessary dynamics of the underlying data. This could be due to several reasons, such as not including relevant variables, needing higher-order polynomial terms (if underfitting), or the data inherently lacking a polynomial relationship.

About the Residuals: The residuals show how off the predictions are for individual data points. Positive values indicate underestimations by the model, while negative values indicate overestimations. The variability in the residuals you provided (-1.3 to 4.05) highlights inconsistencies in the model's prediction accuracy across different observations.

ANOVA (Analysis of Variance) is a statistical method used to compare the means of three or more samples. For regression, ANOVA can test whether the means of different groups (defined by a categorical variable) differ significantly. This is useful for assessing the overall impact of categorical variables like OS on our dependent variable (auction prices).

In [ ]:

# Example data (Replace with your actual data frame)
os_groups = df['osAndVersion'].astype('category').cat.codes.values
auction_prices = df['sentPrice'].values
# auction_prices = df['bidFloorPrice'].values
# Calculate group means
unique_groups = np.unique(os_groups)
group_means = {group: auction_prices[os_groups == group].mean() for group in unique_groups}

# Calculate overall mean
overall_mean = auction_prices.mean()

# Calculate Between-Group Sum of Squares (SSB)
SSB = sum(len(auction_prices[os_groups == group]) * (mean - overall_mean) ** 2 for group, mean in group_means.items())

# Calculate Within-Group Sum of Squares (SSW)
SSW = sum(sum((auction_prices[os_groups == group] - mean) ** 2) for group, mean in group_means.items())

# Calculate total sum of squares (SST)
SST = sum((auction_prices - overall_mean) ** 2)

# Degrees of freedom
df_between = len(unique_groups) - 1
df_within = len(auction_prices) - len(unique_groups)

# Mean Squares
MSB = SSB / df_between
MSW = SSW / df_within

# F-statistic
F = MSB / MSW

print(f"F-statistic: {F}")


We processed our dataset to investigate the impact of operating system (OS) versions on auction prices, employing a simplified ANOVA approach using numpy. Here's a summary of our steps and findings:

Data Preparation: We started by encoding the OS categorical variable into numeric group identifiers, allowing us to handle the data more effectively in a numerical context. This encoding transformed the OS categories into a format suitable for statistical analysis.

Group Mean Calculation: For each unique OS group, we calculated the mean auction price, setting the stage for comparing these group means against the overall mean auction price. This step was crucial for identifying potential variances in auction prices across different OS versions.

Variance Analysis: We decomposed the total variance in auction prices into between-group and within-group components. This allowed us to assess whether the differences in mean auction prices between OS groups were significant compared to the variation within those groups.

F-Statistic Calculation: We computed the F-statistic as 12.44, which quantifies the ratio of variance between the OS groups to the variance within the OS groups. A higher F-statistic suggests significant differences between the group means.

Interpretation: Our calculated F-statistic indicates that there are statistically significant differences in auction prices among different OS versions. This suggests that the OS version has a measurable impact on auction prices, with some OS versions correlating with higher or lower prices than others.

Considerations: While this analysis provided valuable insights, it's important to remember that our approach was simplified. A more comprehensive analysis might include checks for the assumptions underlying ANOVA, explore other influencing factors, or utilize advanced statistical software for a more nuanced understanding.

Next Steps: Based on our findings, we might consider further investigating how specific OS versions influence auction prices or explore additional variables that could affect auction outcomes. Incorporating more complex models or utilizing specialized statistical software could enhance our analysis and provide deeper insights.
Having identified significant differences among OS version groups in their impact on auction prices through our ANOVA analysis, we now consider the following next steps in our investigation:

Post hoc Testing: Our ANOVA results suggest significant variances among the different OS version groups. To further understand these differences, we are interested in conducting post hoc tests, such as Tukey's HSD. This step would help us pinpoint which specific OS versions differ significantly from each other in terms of auction prices. However, implementing post hoc tests like Tukey's HSD manually in a numpy-only environment is complex and may not be straightforward without the functionalities provided by libraries like scipy or statistical software that readily offer these tests.

Checking ANOVA Assumptions: A foundational part of our analysis involves ensuring that the assumptions underpinning ANOVA are satisfied. These include the normality of residuals and the homogeneity of variances among the groups. Verifying these assumptions is critical; failure to meet them may lead us to consider alternative analytical methods or data transformations to uphold the validity of our conclusions. Directly assessing these assumptions requires statistical tests and visualizations that are challenging to implement accurately with basic tools, underscoring the value of specialized statistical libraries.

Further Modeling: We are also contemplating the exploration of interaction effects and the development of different models for data subsets. Such efforts aim to uncover more nuanced insights into how various factors, in combination with OS versions, influence auction prices. This exploration might involve more complex statistical or machine learning models, which could reveal deeper patterns and relationships within our data.

Utilization of Advanced Tools: While our manual approach to calculating the F-statistic has been insightful, we recognize the limitations of not using comprehensive statistical software or libraries like statsmodels. For more detailed analyses that include automatic checks for ANOVA assumptions, calculation of p-values, and execution of post hoc tests, leveraging these advanced tools is essential. They not only simplify the process but also enhance the reliability and depth of our findings.

In summary, our next steps involve a careful consideration of post hoc analyses to understand specific group differences, a thorough check of the ANOVA assumptions to ensure the robustness of our findings, and further exploratory modeling to delve deeper into the data's complexities. We acknowledge the limitations posed by a numpy-only approach for these advanced analyses and recognize the value of specialized statistical libraries in supporting our ongoing research efforts.

when we calculate an F-statistic of 4.570967425758933 using sentPrice instead of bidFloorPrice, it indicates the result of an ANOVA test where osAndVersion groups are used to predict variations in sentPrice. Here's what the components and the result mean in this context:

Overall Mean: The average of sentPrice across all observations, serving as a reference point to compare each group's mean against.
Group Means: The average sentPrice within each OS version group. These are compared to the overall mean to assess whether significant differences exist between groups.
Between-Group Sum of Squares (SSB): Reflects the variability due to the interaction between the different OS versions. A higher SSB indicates more variability between groups, suggesting that different OS versions might have different average sentPrice values.
Within-Group Sum of Squares (SSW): Captures the variability within each OS version group. If the groups are very different in terms of sentPrice, but there's also a lot of variability within groups, it might be harder to attribute differences directly to the OS version.
Total Sum of Squares (SST): The total variability in the dataset regarding sentPrice.
Degrees of Freedom: df_between represents the number of groups minus one, and df_within is the total number of observations minus the number of groups. These help in adjusting the sums of squares to account for the size of the dataset and the number of groups.
Mean Squares (MSB and MSW): Average variability between groups and within groups, respectively. These are calculated by dividing the sums of squares by their corresponding degrees of freedom.
F-statistic: The ratio of MSB to MSW. An F-statistic greater than 1 suggests that there is more variability between groups than within groups, indicating that the OS versions might affect sentPrice.
In this specific case, an F-statistic of 4.570967425758933 suggests that there are significant differences in sentPrice across different OS versions. However, to understand which specific groups (OS versions) differ from each other, post hoc tests like Tukey's HSD would be needed.

In [ ]:

# Standardize 'sentPrice' and 'bidFloorPrice'
sentPrice_standardized = (df['sentPrice'] - np.mean(df['sentPrice'])) / np.std(df['sentPrice'])
bidFloorPrice_standardized = (df['bidFloorPrice'] - np.mean(df['bidFloorPrice'])) / np.std(df['bidFloorPrice'])
#  One-hot encoding for categorical variable 'osAndVersion'
os_encoded = pd.get_dummies(df['osAndVersion'], drop_first=True).values

# Calculate overall mean of 'sentPrice'
overall_mean = np.mean(df['sentPrice'].values)

# SSW and SSB calculation for a single categorical variable (e.g., 'osAndVersion')
SSW = 0
SSB = 0


In [ ]:
# Correcting the variable name to use 'sentPrice_standardized'
for category in range(os_encoded.shape[1]):  # For each category in 'osAndVersion'
    category_data = sentPrice_standardized[os_encoded[:, category] == 1]
    category_mean = np.mean(category_data)
    SSW += np.sum((category_data - category_mean) ** 2)
    SSB += len(category_data) * (category_mean - overall_mean) ** 2
# Assuming these variables are defined elsewhere in your code
df_between = os_encoded.shape[1] - 1
df_within = len(sentPrice_standardized) - os_encoded.shape[1]
MSB = SSB / df_between
MSW = SSW / df_within
F = MSB / MSW

print(f"F-statistic for OS and Version: {F}")


An F-statistic of 674.1981864061144 is quite large and typically suggests that there's a statistically significant difference between the group means being analyzed. In the context of your analysis on the impact of the operating system version (osAndVersion) on the standardized auction prices (sentPrice), this F-statistic would indicate a strong relationship between the OS version and the auction prices.

Such a high F-statistic suggests that at least one OS version group has a mean significantly different from the others when it comes to auction prices. This could be an indication that certain OS versions are associated with higher or lower auction prices compared to the overall average.

However, to fully interpret the F-statistic, you would also need to consider the degrees of freedom associated with the numerator (between groups) and the denominator (within groups), as well as the p-value associated with the F-statistic. The p-value would tell you the probability of observing such a large F-statistic if in reality, there was no difference between the group means (null hypothesis). A commonly used threshold for statistical significance is a p-value of less than 0.05.

In [ ]:

# One-hot encoding for categorical variable 'osAndVersion'
os_encoded = pd.get_dummies(df['osAndVersion'], drop_first=True).values
# 'sentPrice' data
sentPrice = df['sentPrice'].values
# 'bidFloorPrice' data
bidFloorPrice = df['bidFloorPrice'].values

# Number of categories in 'osAndVersion' (after one-hot encoding and dropping first category)
num_categories = os_encoded.shape[1] 

# Number of observations
n_obs = len(sentPrice) 

# Degrees of freedom between (df_between)
df_between = num_categories - 1

# Degrees of freedom within (df_within)
df_within = n_obs - num_categories

# Total degrees of freedom
df_total = n_obs - 1

print(f"Degrees of Freedom Between: {df_between}")
print(f"Degrees of Freedom Within: {df_within}")
print(f"Total Degrees of Freedom: {df_total}")


calculationg the exsexct p value without scipy is quite complex but for this task I thnk we can a conceptual example that may not perform well due to the complexity of accurately calculating F-distribution properties through numerical methods. 
Real-world applications should rely on established libraries like scipy.stats for such tasks, which offer functions like scipy.stats.f.cdf to directly compute p-values from F-statistics efficiently and accurately.

In [ ]:
import math

def f_pdf(x, dfn, dfd):
    """
    Probability density function of the F-distribution.
    Attempting a more stable computation.
    """
    try:
        # Adjusted to use the 'math' module and attempting to stabilize with logs
        log_numerator = math.log(math.gamma((dfn + dfd) / 2)) + ((dfn / 2) - 1) * math.log(dfn / dfd) + ((dfn / 2) - 1) * math.log(x)
        log_denominator = math.log(math.gamma(dfn / 2)) + math.log(math.gamma(dfd / 2)) + ((dfn + dfd) / 2) * math.log(1 + (dfn / dfd) * x)
        return np.exp(log_numerator - log_denominator)
    except OverflowError:
        # If overflow, return a large number; this is not accurate but allows the code to continue.
        return float('inf')

def trapezoidal_rule(f, a, b, n, **kwargs):
    """
    Numerically approximate the integral of f from a to b using the trapezoidal rule with n intervals.
    """
    x = np.linspace(a, b, n+1)
    y = np.array([f(xi, **kwargs) for xi in x])
    h = (b - a) / n
    return (h / 2) * np.sum(y[:-1] + y[1:])

def f_cdf(f, dfn, dfd):
    """
    Approximate the CDF of the F-distribution at point f for dfn and dfd degrees of freedom using numerical integration.
    """
    # Integration bounds and intervals
    a, b, n = 0, f, 10000  # Adjust 'a' and 'n' based on desired accuracy and computational limitations
    return trapezoidal_rule(f_pdf, a, b, n, dfn=dfn, dfd=dfd)


dfn = df_between  # Degrees of freedom numerator (between-groups)
dfd =df_within # Degrees of freedom denominator (within-groups)
f_statistic = F  # Example F-statistic

# Calculate the p-value (right-tail)
p_value = 1 - f_cdf(f_statistic, dfn, dfd)
print(f"P-value: {p_value}")


The result "P-value: -inf" indicates that the approach to compute the p-value for the F-statistic using numerical integration has encountered numerical stability issues, resulting in an overflow that leads to an infinite negative value. This outcome highlights the challenges of manually calculating the cumulative distribution function (CDF) of the F-distribution for extreme values or large datasets without using specialized numerical libraries designed to handle such computations. the original code I tried :
def f_pdf(x, dfn, dfd):
    """
    Probability density function of the F-distribution.
    """
    numerator = (np.math.gamma((dfn + dfd) / 2) * (dfn / dfd) ** (dfn / 2) * x ** ((dfn / 2) - 1))
    denominator = (np.math.gamma(dfn / 2) * np.math.gamma(dfd / 2) * (1 + (dfn / dfd) * x) ** ((dfn + dfd) / 2))
    return numerator / denominator

def f_cdf(f, dfn, dfd):
    """
    Numerically approximate the CDF of the F-distribution for given F-statistic and degrees of freedom.
    """
    # Define the trapezoidal rule function for numerical integration
    def trapezoidal_rule(f, a, b, n, **kwargs):
        x = np.linspace(a, b, n+1)
        y = f(x, **kwargs)
        h = (b - a) / n
        return (h / 2) * np.sum(y[:-1] + y[1:])
    
    # Integration bounds and intervals
    a, b, n = 0, f, 10000  # Adjust 'b' and 'n' as needed
    return trapezoidal_rule(f_pdf, a, b,
resulted in he OverflowError typically happens when the numbers involved in the calculation exceed the maximum limit that Python's floating-point arithmetic can handle. sadly as for submiting I couldn't find a way to fix on problem without getting one error for another.. so I want atempt to reimplament Scipy in the seemly close future =] n, dfn=dfn, dfd=dfd)


The analysis conducted in this notebook provides insightful revelations into the dynamics influencing auction prices within mobile app advertising. Through rigorous statistical exploration, including regression models and ANOVA, we uncovered that app description similarities have a marginal impact on auction prices, as evidenced by R-squared values close to zero. This outcome suggests that the predictive capability of app description similarity on auction prices is minimal. Further, the ANOVA analysis highlighted significant variances in auction prices across different OS versions, pointing towards the operating system as a noteworthy factor in auction price determination.

Despite these insights, the overall low explanatory power of the developed models underscores the presence of other, unaccounted-for factors or more complex relationships that influence auction prices beyond the scope of app descriptions and OS versions. The findings invite additional inquiries into more variables, sophisticated modeling techniques, and deeper analysis of interactions among the factors at play.

In conclusion, while the initial hypotheses about the impact of app descriptions and OS versions on auction prices were partially supported, the limited model effectiveness signals a need for broader investigation. The pursuit of understanding the full spectrum of influences on auction prices remains open, suggesting avenues for future research to refine predictive models, incorporate a wider array of variables, and ultimately, unearth more comprehensive insights into the factors driving auction prices in the mobile app ecosystem.

As suggested erlier , I will now try to look for other variables - Exploring the impact of the country as an independent variable on auction prices can provide valuable insights into geographical influences on market dynamics. 
Here's a suggested approach for conducting this analysis that I would do if I had non limited libary acsess:


Start with descriptive statistics of auction prices for different countries. Calculate means, medians, and standard deviations to get an initial sense of how auction prices vary by country.
Visual Exploration:

Use visualizations such as box plots or histograms to compare auction prices across different countries. This can help identify patterns, outliers, or any country-specific trends in auction prices.
Encoding Categorical Data:

Since 'country' is a categorical variable, I'll need to convert it into a format suitable for regression analysis. One-hot encoding is a common approach that transforms categorical data into binary columns, one for each country.
Regression Analysis:

Incorporate the encoded country data into your regression model as independent variables. This will allow us to assess the impact of each country on auction prices while controlling for other factors.
we can consider using both simple linear regression models to assess individual effects and multiple regression models to understand the combined impact of country and other variables on auction prices.
ANOVA for Country Groups:

Conduct an Analysis of Variance (ANOVA) to test if there are significant differences in auction prices between countries. This can complement your regression analysis by providing a statistical test for the impact of country as a categorical variable.
Interaction Terms:

Explore interaction terms between 'country' and other variables, such as OS version or app description similarity. This can reveal if the impact of one variable on auction prices depends on the level of another variable.
Post-hoc Analysis:

If ANOVA indicates significant differences, perform post-hoc tests to identify which specific country pairs differ. This step helps in understanding the pairwise differences highlighted by ANOVA.
Model Evaluation and Validation:

Evaluate the performance of your models using appropriate metrics such as R-squared for regression models and the F-statistic for ANOVA. Additionally, consider cross-validation techniques to assess the model's predictive accuracy and generalizability.
Sensitivity Analysis:

Conduct sensitivity analysis to understand the robustness of your findings. This could involve varying the model specifications or excluding potentially influential outliers.
Further Exploration:

Based on the results, consider exploring additional variables that might interact with the country, such as economic indicators or mobile usage patterns, to deepen your understanding.

In [ ]:
# Assuming 'df' is your DataFrame and it has columns 'country' and 'sentPrice'
grouped = df.groupby('countryCode')['sentPrice']

# Calculate means, medians, and standard deviations for each country
# Calculate means, medians, standard deviations, and counts for each country
means = grouped.mean()
medians = grouped.median()
std_devs = grouped.std()
counts = grouped.count()  # Add count for the number of auctions per country

# Combine these series into a DataFrame for a comprehensive view
descriptive_stats = pd.DataFrame({
    'Mean': means,
    'Median': medians,
    'Standard Deviation': std_devs,
    'Count': counts  # Include the count in your descriptive statistics
})

# Sort or filter the DataFrame based on 'Count' if needed
# descriptive_stats = descriptive_stats.sort_values(by='Count', ascending=False)

print(descriptive_stats)


aving many countries with only one data point and consequently receiving NaN for the standard deviation (and possibly other statistics) can indeed impact the models and analyses in several ways:

Descriptive Statistics
For descriptive statistics, standard deviation NaN values indicate insufficient data to describe variability. In this context, it means that any measure of dispersion or variability for those countries is undefined or not reliable.

Regression Analysis
In regression analysis, especially if you're using one-hot encoding for countries, countries with very few data points can lead to issues like:

Overfitting: The model might fit too closely to the few data points available for some countries, capturing noise rather than the underlying data pattern. This reduces the model's generalizability.
Coefficient Instability: Regression coefficients associated with countries having very few observations might be unstable and unreliable. Small changes in the data could lead to large changes in the coefficient estimates.
Predictive Power: The model’s ability to predict auction prices for countries with sparse data might be poor due to the lack of sufficient training data for those categories.
ANOVA
For ANOVA, having groups (in this case, countries) with only one observation can be problematic because:

ANOVA Assumptions: One of the assumptions of ANOVA is that the groups being compared should have approximately equal variances. With only one data point per country for several countries, it's impossible to calculate or compare variances meaningfully.
Statistical Significance Testing: ANOVA tests for differences among group means. If many groups have only a single observation, it limits the test's ability to detect significant differences due to a lack of within-group variability information.
Interaction Terms
When considering interaction terms involving countries, the concerns mentioned above are compounded. Interaction terms can increase model complexity, exacerbating issues with overfitting and making interpretation more challenging, especially with sparse data for many countries.

Mitigation Strategies
To mitigate these issues, you might consider:

Aggregating Sparse Categories: Combine countries with very few data points into a broader category, such as "Other."
Focusing on Well-Represented Countries: Limit your analysis to countries with a sufficient number of observations to ensure statistical reliability.
Regularization Techniques: For regression models, consider using regularization (like Lasso or Ridge regression) to handle overfitting and stabilize coefficient estimates.
In summary, while NaN for standard deviation directly reflects on descriptive statistics, it also flags potential concerns for more complex models. Careful preprocessing and model choice can help address these challenges.

Addressing the issue of countries with only one data point in your dataset involves making trade-offs between the granularity of your analysis and the statistical robustness of your findings. Here are some approaches to consider:

Removing Sparse Countries
Pros:

Simplicity: Easy to implement.
Model Stability: Reduces the risk of overfitting and ensures that the model's findings are based on well-represented groups.
Cons:

Loss of Data: Potentially valuable information about those countries is discarded.
Reduced Coverage: The analysis might not reflect the full diversity of the dataset, especially if many countries are removed.
Grouping Sparse Countries into "Other"
Pros:

Inclusivity: Retains information from countries with sparse data by grouping them into a single category.
Reduced Overfitting: Aggregating into an "Other" category helps mitigate the risk of overfitting associated with many small groups.
Statistical Robustness: Enhances the reliability of statistical tests and models by ensuring sufficient sample sizes.
Cons:

Loss of Specificity: Specific insights about individual countries with few data points are lost.
Assumption of Homogeneity: Assumes that all countries grouped into "Other" are similar enough, which might not be accurate.
Using Hierarchical or Mixed-Effects Models
Pros:

Flexibility: Allows for modeling both fixed effects (common to all groups) and random effects (varying between groups), which can account for the variability among countries with more or fewer data points.
Efficiency: Can handle unbalanced designs well, making efficient use of all available data without needing to discard or aggregate sparse groups.
Cons:

Complexity: More complex to set up and interpret than traditional regression models.
Computational Intensity: Typically requires more computational resources and more sophisticated statistical software.
Bayesian Methods
Pros:

Incorporation of Prior Knowledge: Allows for the integration of prior information or beliefs about the data, which can be particularly useful for countries with limited data.
Flexibility in Handling Sparse Data: Can provide more robust estimates for groups with few observations by borrowing strength from the overall data distribution.
Cons:

Complexity: Requires a good understanding of Bayesian statistics and might necessitate specialized software.
Computation Time: Often computationally intensive, especially for large datasets or complex models.

In [ ]:
data_original = df.copy()
country_counts = df['countryCode'].value_counts()
data_removed_sparse = df[df['countryCode'].isin(country_counts[country_counts > 1].index)].copy()
threshold = 2  # You can adjust this threshold based on your dataset
data_grouped_other = df.copy()
sparse_countries = country_counts[country_counts < threshold].index
data_grouped_other['countryCode'] = df['countryCode'].apply(lambda x: 'Other' if x in sparse_countries else x)
# This step is more about ensuring your data is ready for such models rather than creating a new data variable.
data_for_advanced_models = df.copy()
data_for_advanced_models['countryCode'] = data_for_advanced_models['countryCode'].astype('category').cat.codes


# Placeholder function names for each analysis type
def perform_descriptive_stats(data):
    # Your code for calculating descriptive statistics goes here
    return {"Mean": data['sentPrice'].mean(), "Median": data['sentPrice'].median()}

import numpy as np

def perform_regression_analysis(data):
    # Ensure 'countryCode' is numerically encoded if it isn't already
    if data['countryCode'].dtype == 'object':
        # Convert to categorical type then to category codes if it's still in object (string) format
        data['countryCode_encoded'] = data['countryCode'].astype('category').cat.codes
        X = data['countryCode_encoded'].values.reshape(-1, 1)  # Use the encoded column for X
    else:
        # Assume 'countryCode' is already appropriately encoded for numerical operations
        X = data['countryCode'].values.reshape(-1, 1)

    y = data['sentPrice'].values
    
    # Adding a column of ones to include the intercept term in the model
    X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
    
    # Performing the linear regression using the Normal Equation
    beta = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y
    
    # Predicting y values using the regression coefficients
    y_pred = X_with_intercept @ beta
    
    # Calculating residuals
    residuals = y - y_pred
    
    # Calculating Total Sum of Squares (TSS) and Residual Sum of Squares (RSS)
    TSS = np.sum((y - np.mean(y)) ** 2)
    RSS = np.sum(residuals ** 2)
    
    # Calculating R-squared
    R_squared = 1 - (RSS / TSS)
    
    return {"R_squared": R_squared}



def perform_anova(data, group_col='countryCode', outcome_col='sentPrice'):
    # Check if 'countryCode' needs encoding and do so if required
    if group_col not in data.columns:
        data['countryCode_encoded'] = data['countryCode'].astype('category').cat.codes
        group_col = 'countryCode_encoded'
    
    # Proceed with ANOVA using possibly updated group_col
    groups = np.unique(data[group_col])
    overall_mean = np.mean(data[outcome_col])
    
    SSB = 0
    SSW = 0
    for group in groups:
        group_data = data[data[group_col] == group][outcome_col]
        group_mean = np.mean(group_data)
        SSB += len(group_data) * (group_mean - overall_mean) ** 2
        SSW += np.sum((group_data - group_mean) ** 2)
    
    df_between = len(groups) - 1
    df_within = len(data) - len(groups)
    
    MSB = SSB / df_between
    MSW = SSW / df_within
    
    F_statistic = MSB / MSW
    
    return {"F_statistic": F_statistic, "df_between": df_between, "df_within": df_within}




data_versions = {
    "Original": data_original,
    "Removed Sparse Countries": data_removed_sparse,
    "Grouped Other Countries": data_grouped_other,
    "For Advanced Models": data_for_advanced_models
}


# Dictionary to store results
results = {}

for name, data_version in data_versions.items():
    # Perform descriptive stats
    stats = perform_descriptive_stats(data_version)
    
    # Perform regression analysis
    regression_results = perform_regression_analysis(data_version)
    
    # Perform ANOVA analysis, now just directly calling perform_anova without worrying about the column name
    anova_results = perform_anova(data_version)
    
    # Store combined results in the dictionary
    results[name] = {
        "Descriptive Stats": stats,
        "Regression": regression_results,
        "ANOVA": anova_results
    }



# access to the results
for version, analysis_results in results.items():
    print(f"Results for {version}:")
    for analysis_type, result in analysis_results.items():
        print(f"\t{analysis_type}: {result}")


The results from the analysis across different data versions—Original, Removed Sparse, Grouped Other, and Advanced Models—reveal some consistent and some varying trends in the auction prices based on country.

Descriptive Statistics show that the mean auction price is fairly consistent across all data versions, hovering around 1.33 with a median of 0.07. This indicates that while the average price is relatively low, there's a wide range of auction prices, as suggested by the mean being significantly higher than the median.

Regression Analysis yielded R-squared values around 0.01 for all data versions, suggesting that the country variable alone explains only about 1% of the variance in auction prices. This low percentage implies that while there is some relationship between country and auction prices, other factors not accounted for in this analysis likely play a significant role in determining auction prices.

ANOVA Results showed F-statistics ranging from 8.65 to 9.99 across the data versions, with the Removed Sparse and Grouped Other data versions showing slightly higher F-statistics than the Original and Advanced Models versions. The F-statistic measures the ratio of variance between countries to the variance within countries, and these results suggest there are statistically significant differences in auction prices between countries. However, the practical significance of these differences might be limited given the low R-squared values observed in the regression analysis.

In summary, the analysis indicates that while there are statistically significant differences in auction prices among countries, the overall impact of country on auction prices is relatively small, explaining only a small fraction of the variation in prices. Other variables and factors not included in this analysis are likely to have a more substantial impact on auction prices. The slight differences in F-statistics between data versions suggest that removing sparse countries or grouping them into an "Other" category can slightly alter the statistical significance of country differences, but the overall conclusion remains the same.

In [ ]:

# Making 'countryCode' and 'osAndVersion' categorical
data_grouped_other['countryCode'] = data_grouped_other['countryCode'].astype('category')
data_grouped_other['osAndVersion'] = data_grouped_other['osAndVersion'].astype('category')

# Encoding the categorical variables numerically
data_grouped_other['countryCode_encoded'] = data_grouped_other['countryCode'].cat.codes
data_grouped_other['osAndVersion_encoded'] = data_grouped_other['osAndVersion'].cat.codes

# Prepare your features (X) and target (y) for regression
# Let's use 'countryCode_encoded' and 'osAndVersion_encoded' as part of our features
X = data_grouped_other[['countryCode_encoded', 'osAndVersion_encoded']].values
y = data_grouped_other['sentPrice'].values

# Since we can't use KNN due to library constraints, let's consider a simple linear regression as an alternative:
# Adding a column of ones to include the intercept term in the model
X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])

# Performing the linear regression using the Normal Equation
beta = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y

# Predicting y values using the regression coefficients
y_pred = X_with_intercept @ beta

# Calculating residuals
residuals = y - y_pred

# Calculating Total Sum of Squares (TSS) and Residual Sum of Squares (RSS)
TSS = np.sum((y - np.mean(y)) ** 2)
RSS = np.sum(residuals ** 2)

# Calculating R-squared
R_squared = 1 - (RSS / TSS)

print(f"R_squared: {R_squared}")


In [ ]:
def knn_regression(X_train, y_train, X_test, k):
    """
    K-Nearest Neighbors regression.

    Parameters:
    - X_train: Feature set for training data.
    - y_train: Target values for training data.
    - X_test: Feature set for data to predict.
    - k: Number of nearest neighbors to consider.

    Returns:
    - Predictions for X_test.
    """
    # Ensure inputs are numpy arrays for efficient computation
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    
    # Placeholder for predictions
    predictions = np.zeros(X_test.shape[0])

    # Iterate over each item in X_test to make predictions
    for i, test_point in enumerate(X_test):
        # Calculate Euclidean distances from this test point to all training points
        distances = np.sqrt(np.sum((X_train - test_point) ** 2, axis=1))
        
        # Get indices of k smallest distances
        nearest_neighbor_ids = np.argsort(distances)[:k]
        
        # Average the target values of nearest neighbors
        predictions[i] = np.mean(y_train[nearest_neighbor_ids])
        
    return predictions

# def knn_regression(X_train, y_train, X_test, k):
#     """
#     K-Nearest Neighbors regression.

#     Parameters:
#     - X_train: Feature set for training data.
#     - y_train: Target values for training data.
#     - X_test: Feature set for data to predict.
#     - k: Number of nearest neighbors to consider.

#     Returns:
#     - Predictions for X_test.
#     """
#     # Calculate pairwise Euclidean distances between training and test points
#     distances = np.sqrt(np.sum((X_train[:, np.newaxis] - X_test) ** 2, axis=2))
    
#     # Get indices of k smallest distances for each test point
#     nearest_neighbor_ids = np.argsort(distances, axis=0)[:k]
    
#     # Average the target values of nearest neighbors for each test point
#     predictions = np.mean(y_train[nearest_neighbor_ids], axis=0)
    
#     return predictions
 
# Let's split data_grouped_other into a simple train-test split for demonstration
np.random.seed(42)  # For reproducibility
shuffle_indices = np.random.permutation(np.arange(len(y)))
split_idx = int(len(y) * 0.8)  # 80% train, 20% test split

X_train = X_with_intercept[shuffle_indices][:split_idx]
y_train = y[shuffle_indices][:split_idx]
X_test = X_with_intercept[shuffle_indices][split_idx:]
y_test = y[shuffle_indices][split_idx:]

# Perform KNN regression
# Function to calculate MSE
def calculate_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Range of k values to try
# k_values = range(1, 21)  # For example, trying k from 1 to 20
k_values = range(4, 10) 
# Store MSE for each k
mse_values = []

for k in k_values:
    print(k)
    y_pred = knn_regression(X_train, y_train, X_test, k)
    mse = calculate_mse(y_test, y_pred)
    mse_values.append(mse)

# Find the optimal k (with the lowest MSE)
optimal_k = k_values[np.argmin(mse_values)]
optimal_mse = min(mse_values)

print(f"Optimal k: {optimal_k} with MSE: {optimal_mse}")

In [ ]:
optimal_k_predictions = knn_regression(X_train, y_train, X_test, optimal_k)

def calculate_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def calculate_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calculate_r_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot
# Calculate metrics for optimal k
rmse = calculate_rmse(y_test, optimal_k_predictions)
mae = calculate_mae(y_test, optimal_k_predictions)
r_squared = calculate_r_squared(y_test, optimal_k_predictions)

print(f"RMSE for optimal k ({optimal_k}): {rmse}")
print(f"MAE for optimal k ({optimal_k}): {mae}")
print(f"R-squared for optimal k ({optimal_k}): {r_squared}")


The improvement in the R-squared value from 0.0139 to 0.0253 when using KNN regression with the optimal k suggests that incorporating the nearest neighbors' information does offer a better fit for predicting auction prices compared to a simple linear regression model. It indicates that the model is capturing a bit more of the variability in auction prices by considering the similarity between observations.

The RMSE and MAE provide insights into the average error magnitude of your predictions:

RMSE (7.37): This value suggests that, on average, the model's predictions are about 7.37 units away from the actual auction prices in the dataset. Since RMSE penalizes larger errors more heavily, this also implies that there are some predictions with substantial errors, but it gives a somewhat balanced view of the overall prediction error.

MAE (1.81): The Mean Absolute Error provides a more direct interpretation of the average prediction error, indicating that, on average, the predictions are off by approximately 1.81 units from the actual values. It's less sensitive to outliers than RMSE, providing a straightforward measure of prediction accuracy without overly penalizing larger errors.

R-squared (0.0253): The Coefficient of Determination, although still relatively low, has nearly doubled from the linear regression model. It suggests that the KNN model, with the specified number of neighbors, is somewhat more effective at capturing the variance in auction prices than a straightforward linear approach.

These results underline the nuanced nature of predicting auction prices and highlight the potential benefits of exploring non-linear and more complex models like KNN regression. The modest improvement in R-squared, while still leaving a lot of variances unexplained, points towards the complex dynamics at play in auction price determination. It suggests that further model refinement, possibly incorporating additional features or exploring more advanced modeling techniques, could yield even better insights and predictive performance.


In [ ]:
# Assuming auctions_df has 'bundleId' and 'sentPrice', and similarity_df has bundleIds as both index and columns

# Merging the similarity scores with the sentPrice on bundleId
merged_df = pd.merge(similarity_df.reset_index(), auctions_df, on='bundleId', how='inner')

# Before calculating the average similarity, make sure to exclude any non-numeric columns.
numeric_columns = merged_df.select_dtypes(include=[np.number]).columns.tolist()

# Now you can safely drop 'bundleId' if it's not numeric and calculate the mean
merged_df['average_similarity'] = merged_df[numeric_columns].mean(axis=1)

# Calculate the correlation between 'average_similarity' and 'sentPrice'
correlation = merged_df[['average_similarity', 'sentPrice']].corr().iloc[0, 1]
print("Correlation between description similarity and sentPrice:", correlation)

print("Correlation between average description similarity and sentPrice:", correlation)



In [ ]:
merged_df.head()

In [ ]:
# Descriptive statistics for numerical columns
descriptive_stats = merged_df.describe()

# Value counts for each column (typically for categorical data)
value_counts_per_column = {col: merged_df[col].value_counts() for col in merged_df.columns if merged_df[col].dtype == 'object'}

# Number of unique values for each column
unique_values_per_column = merged_df.nunique()

# Data types of each column
column_data_types = merged_df.dtypes

print("Descriptive Statistics:\n", descriptive_stats)
print("\nValue Counts per Column:")
for col, counts in value_counts_per_column.items():
    print(f"\nValue counts for {col}:")
    print(counts)

print("\nUnique Values per Column:\n", unique_values_per_column)
print("\nColumn Data Types:\n", column_data_types)


In [ ]:
from datetime import datetime, timezone


# 1. Remove 'deviceId' and 'bundleId' columns
data = merged_df.drop(['deviceId', 'bundleId'], axis=1)

# 2. Categorize 'countryCode' with threshold
country_counts = data['countryCode'].value_counts()
data['countryCode'] = data['countryCode'].apply(lambda x: x if country_counts[x] > 1 else 'Other')

# 3. Categorize 'brandName' with threshold
brand_counts = data['brandName'].value_counts()
data['brandName'] = data['brandName'].apply(lambda x: x if brand_counts[x] > 1 else 'Other')

# 4. Transform 'osAndVersion'
data['OS_Type'] = data['osAndVersion'].apply(lambda x: 'iOS' if 'iOS' in x else ('Android' if 'Android' in x else 'Other'))
version_counts = data['osAndVersion'].value_counts()
data['OS_Version'] = data['osAndVersion'].apply(lambda x: x if version_counts[x] > 1 else f"{data['OS_Type']} Other")
data = data.drop('osAndVersion', axis=1)

# 5. Transform 'eventTimestamp'
data['eventTimestamp'] = pd.to_datetime(data['eventTimestamp'], unit='ms', utc=True)
data['Weekday'] = data['eventTimestamp'].dt.weekday
data['Hour'] = data['eventTimestamp'].dt.hour
data['Is_Weekend'] = data['Weekday'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

# Mapping numbers to actual day names
data['Day_of_Week'] = data['eventTimestamp'].dt.day_name()

# Dropping the 'eventTimestamp' as it's no longer needed
data = data.drop('eventTimestamp', axis=1)

# Ensuring the 'countryCode' and 'brandName' columns are of category type
data['countryCode'] = data['countryCode'].astype('category')
data['brandName'] = data['brandName'].astype('category')


data.head()

In [ ]:
# Converting 'countryCode' into numerical categories
data['countryCode'] = pd.Categorical(data['countryCode'])
data['countryCode'] = data['countryCode'].cat.codes

# Converting 'brandName' into numerical categories
data['brandName'] = pd.Categorical(data['brandName'])
data['brandName'] = data['brandName'].cat.codes

# Converting 'OS_Type' into numerical categories
data['OS_Type'] = pd.Categorical(data['OS_Type'])
data['OS_Type'] = data['OS_Type'].cat.codes

# Converting 'OS_Version' into numerical categories
data['OS_Version'] = pd.Categorical(data['OS_Version'])
data['OS_Version'] = data['OS_Version'].cat.codes

# For 'Day_of_Week' use a mapping from day names to numbers
day_mapping = {day: i for i, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])}
data['Day_of_Week'] = data['Day_of_Week'].map(day_mapping)

# For 'Is_Weekend' convert to binary numerical category
weekend_mapping = {'Weekday': 0, 'Weekend': 1}
data['Is_Weekend'] = data['Is_Weekend'].map(weekend_mapping)

columns_to_scale = ['average_similarity', 'sentPrice','bidFloorPrice'] #Normalization is typically applied to continuous data, not categorical data, unless those categories have a meaningful order 
for column in columns_to_scale:
    data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

data['unitDisplayType'] = data['unitDisplayType'].astype('category')
data['unitDisplayType'] = data['unitDisplayType'].cat.codes
data.head()

In [ ]:

def knn_regression(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test.to_numpy():
        # Reshape test_point for subtraction
        test_point = test_point.reshape(1, -1)

        # Calculate distances
        distances = np.linalg.norm(X_train.to_numpy() - test_point, axis=1)

        # Find the indices of the k nearest neighbors
        k_indices = np.argpartition(distances, k)[:k]

        # Average the target values of the nearest neighbors
        predictions.append(np.mean(y_train.iloc[k_indices]))
    
    return predictions


def manual_train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    shuffled_indices = np.random.permutation(len(X))
    test_set_size = int(len(X) * test_size)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

def mean_squared_error_manual(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2_score_manual(y_true, y_pred):
    total_variance = np.var(y_true)
    explained_variance = np.var(y_true - y_pred)
    return 1 - (explained_variance / total_variance)





# Split the data into X and y
X = data.drop('sentPrice', axis=1)  # Features
y = data['sentPrice']  # Target variable
X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)

# Convert the first row (predict_point) to a 2D array or ensure it's in the correct shape
predict_point = X.iloc[0].values.reshape(1, -1)

# Use the rest of the dataset for prediction
features = X.iloc[1:]  # excluding the first point to avoid zero distance
targets = y.iloc[1:]  # excluding the corresponding target

mse_scores = []
r2_scores = []
k_values = range(1, 21)  # Example: trying k from 1 to 20

for k in k_values:
    y_pred = knn_regression(X_train, y_train, X_test, k)
    mse = mean_squared_error_manual(y_test, y_pred)
    r2 = r2_score_manual(y_test, y_pred)
    
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"k={k}: MSE={mse}, R^2={r2}")



To summarize the journey and analysis we've embarked on, focusing on enhancing our KNN regression model and exploring the relationship between app description similarity and sent price, we've made significant progress and uncovered interesting findings. Our methodology was constrained by the tools available—namely, NumPy and Pandas—limiting our ability to employ external libraries for machine learning, visualization, and advanced statistical analysis. Despite these constraints, our approach was methodical and insightful, leveraging the core capabilities of these libraries to their fullest.

Summary of Changes and Findings:
Introduction of Embeddings and Cosine Similarity: We started by introducing text embeddings and calculating cosine similarity to measure the similarity between app descriptions. This innovative approach allowed us to capture nuanced semantic relationships that traditional numerical and categorical data might overlook.

Normalization and Preprocessing: Recognizing the importance of normalization in distance-based algorithms like KNN, we applied column-wise normalization to our features. This step ensured that no single feature would disproportionately influence the model due to scale differences.

Optimal K Identification: Through a meticulous process of evaluating different values of k, we determined that k=6 provided the best balance, yielding the lowest mean squared error (MSE) of 54.3890041103897 for our initial set of features. This finding underscored the importance of tuning hyperparameters, even in relatively simple models like KNN.

Feature Engineering and Selection: We undertook feature engineering to transform and categorize our data further. This included converting the osAndVersion column into more granular and informative features and categorizing countries and brands with low occurrence under a common "other" category to reduce sparsity and potential overfitting.

Comparing Analysis and Identifying Future Directions:
Comparing our initial analysis with the subsequent steps involving embeddings and similarity scores, it's clear that incorporating text data and semantic analysis can add valuable dimensions to our model. The initial MSE provided a benchmark, and while our exploration into embeddings and similarity scores introduced complexity, it also opened avenues for deeper understanding and more nuanced predictions.

Future Directions:
Feature Relevance Evaluation: Without access to plotting libraries or advanced statistical packages, we can still delve deeper into feature relevance through correlation analysis and manual inspection. Identifying and potentially removing irrelevant features could streamline the model and improve performance.

Cross-validation: Implementing a manual cross-validation process could provide a more robust evaluation of our model's performance across different subsets of the data, reducing the variance of our MSE estimate and offering insights into the model's stability.

Combining Models: While outside the scope of our current toolset, considering ensemble methods or combining predictions from models trained on different subsets of features (textual similarity versus traditional numerical/categorical data) could yield improvements in accuracy.

Textual Data Exploration: Further refinement of the embedding process or exploring different techniques for generating text embeddings could enhance the model's ability to capture the nuances of app descriptions.

considering the constraints and the progress we've made, focusing on feature importance and selection through methods like correlation and mutual information is a prudent step forward. Let's delve into how these approaches could be incorporated into our analysis and future steps:

Feature Importance via Correlation:
Correlation Analysis: Using Pearson correlation coefficients, we can identify how each feature is linearly related to the target variable (sentPrice). Features with very low correlation coefficients might be considered less important and potentially excluded from the model to improve performance and reduce complexity.
Heatmaps for Visualization: Although our current constraints limit the use of external libraries for visualization, under different circumstances, heatmaps would be an excellent tool for visually identifying relationships between all variables at once.
Feature Selection via Mutual Information:
Mutual Information (MI): This technique measures the amount of information one can obtain from one variable through another. Unlike correlation, MI can capture non-linear relationships, making it a powerful tool for feature selection, especially for complex datasets where linear relationships might not encapsulate the full picture.
Implementation: We can manually implement a simplified version of mutual information calculation using NumPy and Pandas. This would involve discretizing continuous features and employing entropy-based measures to evaluate the dependency between each feature and the target variable.
Future Steps with Feature Importance and Selection:
Iterative Feature Elimination: Based on the insights from correlation and mutual information analyses, we can iteratively remove the least important features and evaluate how the model's performance changes. This approach helps in identifying a minimal set of features that still yields an optimal or near-optimal prediction accuracy.
Exploring Non-linear Relationships: Given the limitations of linear correlation analysis, applying mutual information could unveil hidden patterns and dependencies in the data, leading to more informed feature selection and potentially better model performance.
Practical Considerations:
Manual Calculations and Simplifications: Given the tool constraints, we may need to simplify some calculations or manually implement certain metrics. While this approach is more labor-intensive, it allows for deeper understanding and control over the analysis process.
Focus on Scalable Techniques: As we explore these methods, it's essential to consider the scalability of manual implementations, especially for large datasets. Efficient data manipulation and calculation strategies will be crucial.
Incorporating these advanced feature selection techniques could significantly enhance our model's predictive power and efficiency. By systematically evaluating and selecting features based on their relevance to the target variable, we can streamline the model, reduce overfitting, and potentially uncover more nuanced insights into the factors driving app sent prices.


In [ ]:
#let start with correlation:
correlation_matrix = data.corr()

# Extract correlations with 'sentPrice', excluding the correlation of 'sentPrice' with itself
sentPrice_correlations = correlation_matrix['sentPrice'].drop(['sentPrice','id')

# Sort the correlations by their absolute values in descending order
sorted_correlations = sentPrice_correlations.abs().sort_values(ascending=False)

print(sorted_correlations)


From this correlation analysis with sentPrice, we can glean several insights about the relationship between your features and the target variable. Here's a summary of key takeaways and what they imply for your predictive modeling efforts:

unitDisplayType (0.188479): Shows the highest correlation with sentPrice among all features, suggesting that the type of ad unit significantly influences the price. This could be due to differences in ad effectiveness, visibility, or user engagement across unit types.

bidFloorPrice (0.114992): This is somewhat expected, as the minimum bid price set for an auction might directly influence the final sent price, indicating a positive relationship between the floor price and the final transaction price.

Geographical and Brand Factors: Both countryCode (0.100172) and brandName (0.086067) have notable correlations with sentPrice, indicating that the country in which an ad is displayed and the brand of the device can affect ad pricing. This might reflect market dynamics, purchasing power, or brand preferences.

OS_Type and OS_Version: These factors show a lower degree of correlation but are still notable. They suggest that the operating system type and version might have a small impact on sentPrice, potentially due to differences in user demographics or behavior across different OS platforms.

Embedding Vectors: The embedding vector columns (like 1436213906, se.ace.fishinc, etc.) show varying degrees of correlation, with some being more strongly related to sentPrice than others. This variability suggests that the content or context represented by these embeddings (possibly related to app descriptions or user interests) can have a differential impact on ad pricing.

Temporal Features (Hour, Weekday, Is_Weekend, Day_of_Week): Hour shows a small correlation, suggesting that the time of day might slightly influence ad prices, possibly due to varying user engagement levels throughout the day. However, Weekday, Is_Weekend, and Day_of_Week show very low to no correlation, indicating that the day of the week may not significantly impact sentPrice.

Low Correlation Features: Features like average_similarity have very low correlation values, suggesting they have minimal linear relationship with sentPrice. This doesn't mean they are irrelevant, but they might not contribute significantly to a model that relies on linear relationships.

NaN Values: Is_Weekend and Day_of_Week showing NaN might indicate that these variables have no variation or an issue in computation. If they're constant or nearly constant, they wouldn't contribute to the model's predictive power.

let try now implementing a simplified version of mutual information calculation using NumPy and Pandas:

In [ ]:
def calculate_entropy(y):
    probabilities = y.value_counts(normalize=True)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def calculate_conditional_entropy(x, y):
    # Group the data by feature and calculate the entropy of the target for each group
    conditional_entropy = 0.0
    for value in x.unique():
        y_subset = y[x == value]
        conditional_entropy += (len(y_subset) / len(y)) * calculate_entropy(y_subset)
    return conditional_entropy

def calculate_mutual_information(x, y):
    return calculate_entropy(y) - calculate_conditional_entropy(x, y)

mi_scores = {}
for column in data.columns.drop('sentPrice'):
    if data[column].dtype == 'float64':  # Assuming continuous variables are float64
        data[column] = pd.cut(data[column], bins=10, labels=False)
    mi_scores[column] = calculate_mutual_information(data[column], data['sentPrice'])

# Sort features by MI score
sorted_mi_scores = sorted(mi_scores.items(), key=lambda x: x[1], reverse=True)

for feature, score in sorted_mi_scores:
    print(f"{feature}: {score}")



Mutual information is particularly useful because, unlike correlation, it can capture both linear and non-linear relationships between variables. Here's what we can infer from your results (excluding the id part):

High Mutual Information Scores: Features like countryCode, com.kamilbilge.ropesavior3d, and time-related features (Hour, Weekday, Day_of_Week) have higher mutual information scores. This suggests these features share a significant amount of information with the target variable and are potentially very relevant for predicting sentPrice. For example, countryCode having a high score implies the geographical location of the auction significantly impacts the sent price.

Application and Game Identifiers: Specific application or game identifiers (e.g., com.kamilbilge.ropesavior3d, 1579489488, com.AppIdeas.LevelUpRunner) having high scores indicate that certain applications or games are more predictive of the target variable. This could be due to specific characteristics or popularity of these apps/games affecting the price.

Brand, OS, and Device Attributes: Features like brandName and OS_Version having relatively high mutual information scores suggest that the device brand and the operating system version also play a role in determining sentPrice. This could be tied to user demographics, preferences, or the technical capabilities of devices that influence advertising costs.

Temporal Features: Hour, Weekday, and Day_of_Week showing importance highlights the impact of the timing of auctions on the sent price. This aligns with typical bidding behavior where certain times of the day or week may see higher engagement and therefore higher prices.

Lower Scores for Some Features: Features like OS_Type and bidFloorPrice showing lower mutual information scores might indicate they are less directly related to sentPrice compared to other features. However, they still provide some information and shouldn't be dismissed without further analysis.

Average Similarity: The average_similarity having a certain level of mutual information suggests that the similarity of app descriptions or other textual content might have some impact on the price, though it's not among the top predictors.

My last idea is to use  polynomial and trigonometric feature transformations that can in turn  help capture more complex relationships between features and the target variable, 
sentPrice, potentially improving the performance of KNN regression and informing feature selection through methods like mutual information.

KNN Regression: KNN relies on the proximity of samples in the feature space to make predictions. Non-linear transformations like polynomial and trigonometric functions can change the geometry of the feature space, making patterns more discernible for KNN. This could lead to more accurate predictions, especially if the underlying relationship betw eendata a
�
�
sentPrice is non-linear.

Feature Selection through Mutual Information: Mutual information measures the dependency between variables, capturing both linear and non-linear relationships. By transforming features, you might reveal or amplify underlying patterns that mutual information can detect, leading to a more informed feature selection process. Features that seemed less informative in their original form might show higher mutual information with the target variable after transformation, indicating their hidden relevance.

Feature Elimination: By examining the mutual information or performance of KNN regression before and after feature transformations, you can make more informed decisions about which features to keep or dis �
�
�
�
�
�
�
sentPrice after transformations can be considered for elimination.

In [ ]:
from itertools import combinations_with_replacement
import warnings
warnings.filterwarnings("ignore")

def apply_transformations(df, trig_degree=2, poly_degree=3):
    original_columns = df.select_dtypes(include=[np.number]).columns
    trig_functions = [np.sin, np.cos]

    # Trigonometric Transformations
    for degree in range(1, trig_degree + 1):
        for col in original_columns:
            for func in trig_functions:
                df[f'{func.__name__}{degree}({col})'] = func(degree * df[col])

    # Polynomial/Binomial Expansion-like Feature Combinations
    for degree in range(2, poly_degree + 1):
        # for comb in combinations_with_replacement(df.select_dtypes(include=[np.number]).columns, degree):
        for comb in combinations_with_replacement(original_columns, degree):
            new_col_name = '*'.join(comb)
            df[new_col_name] = df[list(comb)].prod(axis=1)

    return df

# Apply the transformations with specified degrees
features_df = data.drop(['sentPrice','id'], axis=1)  # Exclude the target variable

# Apply transformations to the features DataFrame
transformed_features = apply_transformations(features_df)

transformed_features.tail()


In [ ]:
mi_scores = {}
for column in transformed_features.columns:
    if transformed_features[column].dtype == 'float64':  # Assuming continuous variables are float64
        transformed_features[column] = pd.cut(transformed_features[column], bins=10, labels=False)
    mi_scores[column] = calculate_mutual_information(transformed_features[column], data['sentPrice'])

# Sort features by MI score
sorted_mi_scores = sorted(mi_scores.items(), key=lambda x: x[1], reverse=True)

for feature, score in sorted_mi_scores:
    print(f"{feature}: {score}")


In [ ]:
similarity_bin*Hour*Hour: 1.0378333481904916
similarity_bin*similarity_bin*Hour: 1.0325525965840843
similarity_bin*Weekday*Hour: 0.9154877525415914
similarity_bin*Hour: 0.8476902936790456
Weekday*Hour*Hour: 0.8180888271164592
Weekday*Weekday*Hour: 0.7752092759834417
countryCode: 0.6938422926255212
similarity_bin*similarity_bin*Weekday: 0.6675920436528662
Weekday*Hour: 0.6410068584806856
similarity_bin*Weekday*Weekday: 0.6269447216673258
average_similarity*average_similarity*similarity_bin: 0.5710774608238571
average_similarity*average_similarity_countryCode*similarity_bin: 0.5408681034773108
average_similarity_countryCode*average_similarity_countryCode*similarity_bin: 0.5216506424394636
average_similarity_countryCode*average_similarity_osAndVersion*similarity_bin: 0.5141077561354912
average_similarity_osAndVersion*average_similarity_osAndVersion*similarity_bin: 0.5069109055455385
average_similarity_osAndVersion*similarity_bin: 0.5044684474638119
average_similarity_countryCode*similarity_bin: 0.5041126391077411
average_similarity*average_similarity_osAndVersion*similarity_bin: 0.5036094608715143
similarity_bin: 0.5025044593194421
average_similarity*similarity_bin: 0.5025044593194421
similarity_bin*similarity_bin: 0.5025044593194421
similarity_bin*similarity_bin*similarity_bin: 0.5025044593194421
sin2(average_similarity): 0.499921646906313
sin2(bidFloorPrice): 0.49237474449954544
similarity_bin*Weekday: 0.46623052549888566
cos1(similarity_bin): 0.4626430749839159
average_similarity*average_similarity*average_similarity_osAndVersion: 0.45851103260203363
average_similarity_osAndVersion*similarity_bin*similarity_bin: 0.44809050413166673
average_similarity_countryCode*similarity_bin*similarity_bin: 0.44764521043854266
average_similarity*similarity_bin*similarity_bin: 0.4466140485907433
average_similarity*average_similarity*average_similarity_countryCode: 0.41716399774443236
average_similarity*average_similarity: 0.4170837345544509
average_similarity*average_similarity*average_similarity: 0.4170837345544509
cos1(average_similarity): 0.40197707723416
cos2(average_similarity): 0.40197707723416
average_similarity: 0.39900204158753905
sin1(average_similarity): 0.39900204158753905
Hour: 0.38837384371850714
Hour*Hour: 0.38837384371850714
Hour*Hour*Hour: 0.38837384371850714
average_similarity*average_similarity_osAndVersion: 0.3545405663727381
sin2(similarity_bin): 0.3498810880669936
brandName: 0.3493953262154017
OS_Version: 0.34464456239160857
average_similarity*average_similarity_countryCode: 0.3354092786226426
sin1(similarity_bin): 0.3278253157300153
cos2(similarity_bin): 0.31719786282526385
average_similarity*average_similarity_countryCode*average_similarity_osAndVersion: 0.3046814481102924
unitDisplayType: 0.2928241709638675
average_similarity*average_similarity_osAndVersion*average_similarity_osAndVersion: 0.28680101387959933
average_similarity*average_similarity_countryCode*Weekday: 0.27010508013568657
average_similarity_osAndVersion*similarity_bin*Hour: 0.2698798136996876
average_similarity_countryCode*similarity_bin*Hour: 0.26862176823579276
average_similarity*similarity_bin*Hour: 0.26643763718068136
average_similarity*average_similarity_countryCode*average_similarity_countryCode: 0.2589782353169232
average_similarity*average_similarity*Weekday: 0.2450634843753301
average_similarity_countryCode*similarity_bin*Weekday: 0.24495010662533012
average_similarity*Weekday: 0.24398177276087907
average_similarity*similarity_bin*Weekday: 0.23815678414916164
average_similarity_osAndVersion*similarity_bin*Weekday: 0.23777236302990712
average_similarity*average_similarity*Hour: 0.22553330728193277
average_similarity*Hour: 0.22436931604582533
average_similarity*average_similarity_osAndVersion*Hour: 0.22422792799215774
average_similarity*average_similarity_countryCode*Hour: 0.22415098956899016
average_similarity_osAndVersion*average_similarity_osAndVersion*Hour: 0.22139844062585734
average_similarity_countryCode*average_similarity_countryCode*Hour: 0.22103603480817569
average_similarity_countryCode*Hour: 0.2203687661405782
average_similarity_countryCode*average_similarity_osAndVersion*Hour: 0.22017335273761418
average_similarity*average_similarity_osAndVersion*Weekday: 0.22003662587416262
average_similarity_osAndVersion*Hour: 0.21936276570602065
average_similarity*Hour*Hour: 0.21294308619314517
average_similarity_countryCode*Hour*Hour: 0.2106716201675427
average_similarity_osAndVersion*Hour*Hour: 0.20926591627839564
sin2(Hour): 0.20879483057998893
cos1(Hour): 0.2014318373382311
average_similarity*Weekday*Weekday: 0.19815468292854632
average_similarity_countryCode*average_similarity_countryCode*Weekday: 0.1927068829341385
average_similarity_countryCode*average_similarity_osAndVersion*Weekday: 0.18958297783343614
sin1(Hour): 0.1887628568337405
average_similarity_osAndVersion*Weekday*Hour: 0.18151567356999188
average_similarity_countryCode*Weekday: 0.18087168914877338
average_similarity_countryCode*Weekday*Hour: 0.1799559255818668
average_similarity_osAndVersion*average_similarity_osAndVersion*Weekday: 0.17720694966107242
average_similarity*Weekday*Hour: 0.1764837704768727
average_similarity_osAndVersion*Weekday: 0.1748686872012275
cos2(Hour): 0.17448376835412294
Weekday: 0.17233976268350304
Day_of_Week: 0.17233976268350304
Weekday*Weekday: 0.17233976268350304
Weekday*Weekday*Weekday: 0.17233976268350304
cos1(average_similarity_countryCode): 0.15305937606430575
cos1(Weekday): 0.1514373029344096
average_similarity_countryCode*average_similarity_countryCode: 0.1509785998978952
average_similarity_countryCode*average_similarity_osAndVersion: 0.15086258751587778
average_similarity_osAndVersion*Weekday*Weekday: 0.15011564396370503
average_similarity_countryCode*Weekday*Weekday: 0.14860140505488406
average_similarity_countryCode*average_similarity_countryCode*average_similarity_countryCode: 0.148166231758327
sin1(average_similarity_countryCode): 0.14575222561069445
average_similarity_countryCode: 0.14561665928690193
cos2(average_similarity_countryCode): 0.14547772344680165
average_similarity_countryCode*average_similarity_countryCode*average_similarity_osAndVersion: 0.1304803565349486
sin2(Weekday): 0.12938625187738317
cos2(Weekday): 0.12774275193075546
sin2(average_similarity_countryCode): 0.12385902378829439
sin1(Weekday): 0.12239570861789861
average_similarity_countryCode*average_similarity_osAndVersion*average_similarity_osAndVersion: 0.10065591402386254
sin2(average_similarity_osAndVersion): 0.08804006879264659
average_similarity_osAndVersion: 0.08276992578419673
cos1(average_similarity_osAndVersion): 0.08228346841637446
average_similarity_osAndVersion*average_similarity_osAndVersion: 0.08223105701849942
cos2(average_similarity_osAndVersion): 0.08213984459567047
average_similarity_osAndVersion*average_similarity_osAndVersion*average_similarity_osAndVersion: 0.08184961650700728
sin1(average_similarity_osAndVersion): 0.0790543336395979
OS_Type: 0.0725347994395733
bidFloorPrice*average_similarity_osAndVersion*Hour: 0.05370942905024112
bidFloorPrice*Hour: 0.05348964621640917
bidFloorPrice*bidFloorPrice*Hour: 0.05348964621640917
bidFloorPrice*average_similarity*Hour: 0.05348964621640917
bidFloorPrice*average_similarity_countryCode*Hour: 0.05348964621640917
bidFloorPrice*similarity_bin*Hour: 0.05348964621640917
bidFloorPrice*average_similarity_osAndVersion*average_similarity_osAndVersion: 0.050556214251455955
bidFloorPrice: 0.05037345065875787
sin1(bidFloorPrice): 0.05037345065875787
cos1(bidFloorPrice): 0.05037345065875787
cos2(bidFloorPrice): 0.05037345065875787
bidFloorPrice*bidFloorPrice: 0.05037345065875787
bidFloorPrice*average_similarity: 0.05037345065875787
bidFloorPrice*average_similarity_countryCode: 0.05037345065875787
bidFloorPrice*average_similarity_osAndVersion: 0.05037345065875787
bidFloorPrice*similarity_bin: 0.05037345065875787
bidFloorPrice*bidFloorPrice*bidFloorPrice: 0.05037345065875787
bidFloorPrice*bidFloorPrice*average_similarity: 0.05037345065875787
bidFloorPrice*bidFloorPrice*average_similarity_countryCode: 0.05037345065875787
bidFloorPrice*bidFloorPrice*average_similarity_osAndVersion: 0.05037345065875787
bidFloorPrice*bidFloorPrice*similarity_bin: 0.05037345065875787
bidFloorPrice*average_similarity*average_similarity: 0.05037345065875787
bidFloorPrice*average_similarity*average_similarity_countryCode: 0.05037345065875787
bidFloorPrice*average_similarity*average_similarity_osAndVersion: 0.05037345065875787
bidFloorPrice*average_similarity*similarity_bin: 0.05037345065875787
bidFloorPrice*average_similarity_countryCode*average_similarity_countryCode: 0.05037345065875787
bidFloorPrice*average_similarity_countryCode*average_similarity_osAndVersion: 0.05037345065875787
bidFloorPrice*average_similarity_countryCode*similarity_bin: 0.05037345065875787
bidFloorPrice*average_similarity_osAndVersion*similarity_bin: 0.05037345065875787
bidFloorPrice*similarity_bin*similarity_bin: 0.05037345065875787
bidFloorPrice*Weekday: 0.049388659039258265
bidFloorPrice*bidFloorPrice*Weekday: 0.049388659039258265
bidFloorPrice*average_similarity*Weekday: 0.049388659039258265
bidFloorPrice*average_similarity_countryCode*Weekday: 0.049388659039258265
bidFloorPrice*average_similarity_osAndVersion*Weekday: 0.049388659039258265
bidFloorPrice*similarity_bin*Weekday: 0.049388659039258265
bidFloorPrice*Hour*Hour: 0.0433520189741774
bidFloorPrice*Weekday*Weekday: 0.040159553035404194
bidFloorPrice*Weekday*Hour: 0.038140417206157906
Is_Weekend: 0.037187577852753506

Embarking on this journey, we hypothesized that apps with similar descriptions would exhibit comparable asking prices in auctions, as denoted by the sentPrice column. To explore this, we employed cosine similarity to assess the similarity between app descriptions based on their embeddings and applied various statistical tools to substantiate or refute our hypothesis.

Our initial steps involved calculating mutual information between features and sentPrice using the untransformed dataset, aiming to uncover direct contributions of each feature towards predicting auction prices. Here, we anticipated that embeddings reflecting similar descriptions would show a correlation with akin sentPrice values.

Progressing further, we transformed the dataset, introducing new features through trigonometric and polynomial expansions, targeting to unveil more intricate relationships between the features and the target variable that linear assessments might overlook. This maneuver was predicated on the belief that if apps with analogous descriptions indeed share similar auction prices, these augmented features would elucidate this pattern more conspicuously.

Post-transformation analysis via mutual information unveiled noteworthy insights. Particularly, transformed features frequently exhibited a heightened mutual dependency with sentPrice compared to the original set. This revelation hints at the possibility that the liaison between app description similarities (as encapsulated by the embeddings) and auction prices extends beyond simple linear connections, delving into more complex, potentially non-linear realms. The augmented mutual information scores for specific feature combinations, particularly those entailing transformed attributes, accentuated this complexity, suggesting that a blend of multiple features might yield a more precise prediction of auction prices.

These outcomes intimate that the correlation between app description similarity and auction price transcends straightforward linear associations, involving intricate interactions among various features. Notably, transformed features like similarity_bin*Hour*Hour with a mutual information score of 1.0378333481904916 and similarity_bin*similarity_bin*Hour at 1.0325525965840843, stand out as significant. Their high mutual information values illuminate the intricate dynamics at play in predicting sentPrice, pointing towards the nuanced and multifaceted nature of this relationship.

Reflecting on these insights compels us to reassess our initial hypothesis under a new light. The data suggests a nuanced relationship between app description similarity and auction prices, one that is captured through a complex interplay of both linear and non-linear feature interactions. Thus, our analysis, underpinned by cosine similarity and enriched through the statistical examination of transformed features, lends credence to our hypothesis, albeit revealing the intricate layers that govern the relationship between app description similarity and auction prices.

As we forge ahead, our strategy will involve a meticulous refinement of the feature selection process, informed by the revelations from the mutual information analysis post-transformation. Concentrating on the most informative features—whether original or transformed—promises a path towards devising a more precise model capable of predicting auction prices from app description similarities effectively. This refined approach not only reiterates the potential validity of our hypothesis but also embarks us on an advanced exploration into the myriad factors influencing sentPrice in app auctions, thereby embracing the complexity and richness of the data at our disposal.


its recomended no to run the next cell:

In [ ]:
# Split the data into X and y
X = transformed_features  # Features
y = data['sentPrice']  # Target variable
X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)

# Convert the first row (predict_point) to a 2D array or ensure it's in the correct shape
predict_point = X.iloc[0].values.reshape(1, -1)

# Use the rest of the dataset for prediction
features = X.iloc[1:]  # excluding the first point to avoid zero distance
targets = y.iloc[1:]  # excluding the corresponding target

mse_scores = []
r2_scores = []
k_values = range(16, 21)  # Example: trying k from 16 to 20 

for k in k_values:
    y_pred = knn_regression(X_train, y_train, X_test, k)
    mse = mean_squared_error_manual(y_test, y_pred)
    r2 = r2_score_manual(y_test, y_pred)
    
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"k={k}: MSE={mse}, R^2={r2}")


In [ ]:
k=16: MSE=0.0002507651835986358, R^2=-0.03707803817177879
k=17: MSE=0.0002501477558128505, R^2=-0.034522545479094724
k=18: MSE=0.00024883147666771455, R^2=-0.029076953354937096
k=19: MSE=0.00024833913993367525, R^2=-0.027034616955534885
k=20: MSE=0.0002478244182300384, R^2=-0.02490592114466028

Following the transformation and introduction of new columns into our dataset, we embarked on a re-evaluation of the KNN regression model's performance. Notably, the results post-transformation demonstrated an improvement in the Mean Squared Error (MSE) across different values of �
k, with a minimum MSE observed ak=
2
R 
 of R^22
  values provides insightful revelations into our modeling approach and hypothesis.

We hypothesized that apps with similar descriptions would likely have similar asking prices in the auctions. This hypothesis was predicated on the assumption that cosine similarity, applied to the embeddings of app descriptions, could serve as a predictive indicator of auction prices. The initial analysis aimed to validate this hypothesis by leveraging statistical measures to explore the relationship between app description similarities and auction prices.

The transformation process introduced new features through trigonometric and polynomial transformations, aiming to uncover more complex relationships within the data. This step was based on the assumption that the intrinsic relationship between app descriptions and auction prices might not be linear but could involve more intricate, nonlinear dynamics that our initial model did not fully capture.

The observed improvement in MSE post-transformation suggests that the inclusion of these new features indeed facilitated a more nuanced understanding of the underlying patterns in the data. Specifically, the reduction in MSE indicates that the transformed dataset, with its augmented features, aligns more closely with the actual auction prices, thereby enhancing the model's predictive accuracy.

R^2ment in 
�
2
R 
2
  values, while indicative of a model that does not yet optimally predict auction prices, signals an interesting aspect of our hypothesis testing. The rR^2 eductioe change in 
�
2
R 
2
  values underscore the complexity of the relationship between app descriptions and auction prices. It suggests that while we are moving closer to capturing the essence of this relationship, there remains a nuanced interplay of factors that needs to be unraveled.

In interpreting these results, we acknowledge the pivotal role played by the transformed features in reducing the predictive error of our model. This reinforces our belief in the potential for complex feature interactions to more accurately reflect the dynamics between app descriptions and auction prices. Moving forward, we are encouraged to further refine our feature selection and transformation techniques to enhance the model's explana R^2td achieve positive 
�
2
R 
2
  values, providing a more robust validation of our initial hypothesis.

In summary, the observed improvement in MSE post-feature transformation and creation offers a promising direction for our analysis. It emphasizes the importance of embracing the data's complexity through thoughtful feature engineering, guiding us toward a deeper understanding of the intricate relationship between app description similarities and auction prices.SE and


In this notebook, the analysis focused on investigating the hypothesis that apps with similar descriptions would have similar asking prices in auctions, as represented by the 'sentPrice' column. The key steps and findings are as follows:
Embeddings and Cosine Similarity: The analysis started by introducing text embeddings and calculating cosine similarity to measure the similarity between app descriptions. This approach allowed the capture of nuanced semantic relationships that traditional numerical and categorical data might have overlooked.
Normalization and Preprocessing: The data was normalized to ensure that no single feature would disproportionately influence the model due to scale differences.
KNN Regression Exploration: Despite initial hesitation about the computational complexity of KNN regression for the large dataset, the approach was ultimately implemented. The analysis evaluated different values of k and found that k=6 provided the best balance, yielding the lowest mean squared error (MSE).
Feature Engineering and Selection: The notebook undertook feature engineering, transforming and categorizing the data further, including converting the 'osAndVersion' column into more granular features and handling low-occurrence countries and brands.
Comparing Analyses and Identifying Future Directions: Comparing the initial analysis with the subsequent steps involving embeddings and similarity scores, it became evident that incorporating text data and semantic analysis can add valuable dimensions to the model. However, the overall low explanatory power of the models suggested the presence of other, unaccounted-for factors or more complex relationships influencing auction prices.
Leveraging KNN Regression: The implementation of KNN regression, despite the initial concerns, led to an improvement in the R-squared value from 0.0139 to 0.0253. This suggested that the KNN model was better able to capture the variability in auction prices compared to the simpler linear regression approach.
Future Directions: The analysis pointed towards several future directions, including feature relevance evaluation, cross-validation, combining models, and further exploration of textual data processing techniques to uncover the nuanced relationships between app description similarities and auction prices.
The key takeaway is that the relationship between app description similarity and auction prices is more complex than a simple linear association. The incorporation of transformed features, such as trigonometric and polynomial expansions, revealed heightened mutual information scores, suggesting the presence of intricate, non-linear dynamics governing this relationship.
While the initial hypothesis was partially supported, the limited model effectiveness signaled the need for a broader investigation. The pursuit of understanding the full spectrum of influences on auction prices remains open, suggesting avenues for future research to refine predictive models, incorporate a wider array of variables, and ultimately, unearth more comprehensive insights into the factors driving auction prices in the mobile app ecosystem.